## 🃏 Draw Poker
#### Solomon Himelbloom (2023-12-08)

In [1]:
from src.evaluation import *

In [2]:
print(f">> Standard {len(deck)}-card deck.")

print("Permutations: " + str(len(partial)))

print("Combinations: " + str(len(hands)))

>> Standard 52-card deck.
Permutations: 132600
Combinations: 22100


In [3]:
category_counts = {
    category: 0
    for category in [
        "Royal Flush",
        "Straight Flush",
        "Three Aces",
        "Three of a Kind",
        "Straight",
        "Flush",
        "Pair",
        "High Card",
    ]
}

for hand in hands:
    category = categorize(hand)
    if category != "One Pair":
        category_counts[category] += 1

print(">> Category Frequencies")
for category, count in category_counts.items():
    print(f"{category}: {count}")

print()

print(">> Category Probabilities")
for category, count in category_counts.items():
    print(f"{category}: {count / len(hands)}")

>> Category Frequencies
Royal Flush: 4
Straight Flush: 44
Three Aces: 4
Three of a Kind: 48
Straight: 720
Flush: 1096
Pair: 3744
High Card: 16440

>> Category Probabilities
Royal Flush: 0.00018099547511312217
Straight Flush: 0.001990950226244344
Three Aces: 0.00018099547511312217
Three of a Kind: 0.0021719457013574662
Straight: 0.03257918552036199
Flush: 0.049592760180995475
Pair: 0.16941176470588235
High Card: 0.7438914027149321


### Benchmark Analysis

#### Expected Value

- To make the game more interesting, the player can replace (or draw) 0, 1, 2 or all 3 cards. 
- Determine the perfect play for any 3 card hand. For instance, with As Ah Kh, should you hold As Ah or Ah Kh? There’s 49 cards left and you compute the E[As Ah] and E[Ah Kh] and pick the higher one. 
- To generalize, compute E[x] for all eight possible holds and use the hold with the highest E[x]. Create 10 “interesting” hands and show the E[x] for each hold and which one is the perfect play. Include the output from the test hands.

In [4]:
payouts = {
    "Royal Flush": 250,
    "Straight Flush": 100,
    "Three Aces": 100,
    "Three of a Kind": 30,
    "Straight": 15,
    "Flush": 5,
    "Pair": 1,
    "High Card": 0,
}

In [5]:
samples = [
    ["A of ♠", "A of ♥", "K of ♥"],
    ["J of ♠", "6 of ♦", "8 of ♦"],
    ["A of ♠", "K of ♠", "Q of ♠"],
    ["J of ♠", "10 of ♠", "9 of ♠"],
    ["A of ♠", "A of ♥", "A of ♦"],
    ["7 of ♠", "7 of ♥", "7 of ♦"],
    ["A of ♠", "2 of ♥", "3 of ♦"],
    ["A of ♠", "Q of ♠", "9 of ♠"],
    ["A of ♠", "A of ♥", "K of ♠"],
    ["A of ♠", "8 of ♥", "9 of ♠"],
]

In [6]:
def ev_calc(hold, hand, deck):
    """Calculate the expected value of a hand given the cards to hold."""
    remainder = [card for card in deck if card not in hand]
    likelihoods = {}

    # Probability Outcomes
    for draw in combinations(remainder, size - len(hold)):
        new_hand = list(hold) + list(draw)
        category = categorize(new_hand)
        likelihoods[category] = likelihoods.get(category, 0) + 1

    # Probability Normalization
    total_probability = sum(likelihoods.values())
    for category in likelihoods:
        likelihoods[category] /= total_probability

    return sum(likelihoods[category] * payouts[category] for category in likelihoods)

def ideal_play(hand, deck):
    """Find the ideal play for a given hand."""
    max_ev = 0
    best_hold = None
    replacement = [0, 1, 2, 3]

    for draw in replacement:
        holds = list(combinations(hand, len(hand) - draw))

        for hold in holds:
            ev = ev_calc(list(hold), hand, deck)
            print(f"Hold: {hold}, Expected Value: {ev}")

            if ev > max_ev:
                max_ev = ev
                best_hold = hold

    return best_hold, max_ev

In [7]:
for hand in samples:
    print(f">> Hand: {hand} → {categorize(hand)}")
    print(f">> Ideal Play: {ideal_play(hand, deck)}\n")

>> Hand: ['A of ♠', 'A of ♥', 'K of ♥'] → Pair
Hold: ('A of ♠', 'A of ♥', 'K of ♥'), Expected Value: 1.0
Hold: ('A of ♠', 'A of ♥'), Expected Value: 5.040816326530612
Hold: ('A of ♠', 'K of ♥'), Expected Value: 1.3265306122448979
Hold: ('A of ♥', 'K of ♥'), Expected Value: 7.142857142857142
Hold: ('A of ♠',), Expected Value: 1.125
Hold: ('A of ♥',), Expected Value: 0.8826530612244897
Hold: ('K of ♥',), Expected Value: 0.858843537414966
Hold: (), Expected Value: 1.2608554059921842
>> Ideal Play: (('A of ♥', 'K of ♥'), 7.142857142857142)

>> Hand: ['J of ♠', '6 of ♦', '8 of ♦'] → High Card
Hold: ('J of ♠', '6 of ♦', '8 of ♦'), Expected Value: 0.0
Hold: ('J of ♠', '6 of ♦'), Expected Value: 0.12244897959183673
Hold: ('J of ♠', '8 of ♦'), Expected Value: 0.12244897959183673
Hold: ('6 of ♦', '8 of ♦'), Expected Value: 4.1020408163265305
Hold: ('J of ♠',), Expected Value: 1.346938775510204
Hold: ('6 of ♦',), Expected Value: 1.1811224489795917
Hold: ('8 of ♦',), Expected Value: 1.181122448979

#### Perfect Play

- Determine the perfect play for all hands. Frequency/Probability is no longer meaningful - you will sum the E[x] for the perfect play for each hand to determine the total return. Include the updated return table with perfect play.

In [8]:
total_return = sum(ev_calc(hold, hand, deck) for hold in hands)

| Hand | Description | Payout |
| --- | --- | --- |
| Royal Flush | AKQ (in any suit) | \$250 |
| Straight Flush | 3 suited in sequence | \$100 |
| Three Aces | 3 Aces (any combo of suits) | \$100 |
| Three of a Kind | 3 of the same rank | \$30 |
| Straight | 3 in sequence (includes AKQ) | \$15 |
| Flush | 3 suited | \$5 |
| Pair | 2 of the same rank | \$1 | 
| High Card | None of the above | \$0 | 

In [9]:
print(f">> Perfect Play: {total_return / len(hands)}")

>> Perfect Play: 1.2336651583710407
